<a href="https://colab.research.google.com/github/alyosha-swamy/ACM-Assignment/blob/main/Copy_of_iris_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import plotly.graph_objs as go

# Started with imports
I first browsed the keras/tensorflow implementations for this. I only know pytorch so I chose to use it here. Matplotlib is terrible, I like plotly and I will be using it here for the graphs I will be plotting. Numpy is for processing the data and also arranging it while plotting,

In [ ]:
# Load the iris dataset as a CSV file
data = np.loadtxt('iris.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3))
target = np.loadtxt('iris.csv', delimiter=',', skiprows=1, usecols=(4,), dtype=float)

# Convert the target labels to numerical values
target[target == 'Iris-setosa'] = '0'
target[target == 'Iris-versicolor'] = '1'
target[target == 'Iris-virginica'] = '2'

# Scale the features manually
mean = np.mean(data, axis=0)
std = np.std(data, axis=0)
data_scaled = (data - mean) / std

# Split the data into training and testing sets
np.random.seed(42)
indices = np.random.permutation(len(data_scaled))
train_indices, test_indices = indices[:int(0.5*len(data_scaled))], indices[int(0.5*len(data_scaled)):]
X_train, y_train = data_scaled[train_indices], target[train_indices]
X_test, y_test = data_scaled[test_indices], target[test_indices]


<ipython-input-14-037544a62219>:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  target[target == 'Iris-setosa'] = '0'
<ipython-input-14-037544a62219>:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  target[target == 'Iris-versicolor'] = '1'
<ipython-input-14-037544a62219>:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  target[target == 'Iris-virginica'] = '2'


## Preparing data
 I started with pulling the csv file and arranging it. Here we will use the characteristics of the flowers as inputs to our models. The target is the label of the exact flower, which is labelled with an integer (0, 1, 2) corresponding to the type of flower. The first preprocessing method is described after this, I am scaling it by subtracting the mean and dividing it by the standard deviation. This helps by normalizing the inputs as sometimes when the input features have different scales, some features may have a larger influence on the model's predictions than others, simply because their values are larger. Now the values would be centered around and the variance would be close to one.



# Splitting
Then I split the data into a train and test split for the models. The dataset is split at random to prevent any unwanted bias or overfitting. The seed is taken so that the random number can be reproduced each time. This dataset is split towards 50 percent of the dataset, I find that the results are slightly better when its closer to the middle like this.


In [ ]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

# Pytorch Tensors
Just converting the dataset splits, which were numpy arrays initally, into pytorch tensors for later operations.

In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        logits = self.linear(x)
        return logits

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        z1 = self.fc1(x)
        a1 = self.relu(z1)
        z2 = self.fc2(a1)
        return z2

# Define the models and the optimizer
lr_model = LogisticRegression(input_size=4, num_classes=3)
nn_model = NeuralNetwork(input_size=4, hidden_size=64, num_classes=3)
lr_optimizer = optim.SGD(lr_model.parameters(), lr=0.01)
nn_optimizer = optim.SGD(nn_model.parameters(), lr=0.01)

I am defining two classes here for log reg and the neural network. Going to define the functions inside manually. Logistic regression is a simple one layer linear function, The model takes an input tensor x and multiplies it by a weight tensor W, adds a bias tensor b, and returns the result as the logits. During training we will compute loss W.R.T the weight and bias 'parameters' and then update the gradients during backprop. 

For the neural network, I am using 2 layers(hidden). The input x is first multiplied by the weight matrix W1 and then the bias vector b1 is added. This is z1, which I am passing into a relu activation function, which is sort of a switch (f(x)=max(0,x)). The relu makes computing gradients easier and training is much faster. Then this activation output a1 is then multiplied by the weight matrix W2 and the bias vector b2 is added to produce the final pre-activation output z2, which is returned as the logits.

Defining the models works by just calling the classes with the inputs. Log reg has a very simple process, it takes 4 input features and predicts between 3 possible results. The neural network has a additional 64 neuron hidden layer in the middle between these.
Then I am using the stochastic gradient descent algo for optimizing these values. An optimizer is basically used to update the model's parameters during training, in order to minimize the loss function and improve the model's performance. A common learning rate for such problems is 0.01, I experimented playing around witht his and felt this was optimum. 

In [ ]:
from collections import defaultdict

lr_history = defaultdict(list)
nn_history = defaultdict(list)

Another data processing is done here I use defaultdict instead of regular dict to go over values that might be NULL. It provides a default value for the key that does not exists and thus is slightly better than a regular dictionary.

In [ ]:
# Train the logistic regression model
for epoch in range(1000):
    # Forward pass
    logits = lr_model(X_train)
    loss = nn.functional.cross_entropy(logits, y_train)

    # Backward pass
    lr_optimizer.zero_grad()
    loss.backward()
    lr_optimizer.step()

    # Compute training accuracy
    preds = torch.argmax(logits, axis=1)
    accuracy = torch.mean((preds == y_train).float())

    # Save loss and accuracy for plotting
    lr_history['loss'].append(loss.item())
    lr_history['accuracy'].append(accuracy.item())

# Train the neural network model
for epoch in range(1000):
    # Forward pass
    logits = nn_model(X_train)
    loss = nn.functional.cross_entropy(logits, y_train)

    # Backward pass
    nn_optimizer.zero_grad()
    loss.backward()
    nn_optimizer.step()

    # Compute training accuracy
    preds = torch.argmax(logits, axis=1)
    accuracy = torch.mean((preds == y_train).float())

    # Save loss and accuracy for plotting
    nn_history['loss'].append(loss.item())
    nn_history['accuracy'].append(accuracy.item())

# Defining simple training loops for both the models
I run it over 1000 epochs or runs thru the dataset. First I perform a forward pass of the model using the training data to obtain the predicted outputs (logits). Then compute the loss between the predicted outputs and the true labels (y_train) using the cross entropy loss function. This is a standard loss function given by $-\frac{1}{N}\sum_{i=1}^N\sum_{j=1}^M y_{i,j}\log(p_{i,j})$. It measures the difference between the predicted probability distribution and the true probability distribution. After that, I perform a backward pass through the network to compute the gradients of the loss with respect to the model parameters. Lastly I use the optimizer (lr_optimizer or nn_optimizer) to update the model parameters based on the computed gradients and then compute the training accuracy by comparing the predicted outputs to the true labels, and record the loss and accuracy values for plotting purposes.

In [ ]:
# Smooth the curves using a moving average over 5 points
window_size = 5
weights = np.repeat(1.0, window_size)/window_size
lr_loss_smooth = np.convolve(lr_history['loss'], weights, 'valid')
nn_loss_smooth = np.convolve(nn_history['loss'], weights, 'valid')
lr_acc_smooth = np.convolve(lr_history['accuracy'], weights, 'valid')
nn_acc_smooth = np.convolve(nn_history['accuracy'], weights, 'valid')

# Define the data traces for the loss plot
trace1 = go.Scatter(x=np.arange(window_size//2, len(lr_loss_smooth)+window_size//2), y=lr_loss_smooth, name='Logistic Regression', mode='lines+markers')
trace2 =  go.Scatter(x=np.arange(window_size//2, len(nn_loss_smooth)+window_size//2), y=nn_loss_smooth, name='Neural Network', mode='lines+markers')
data_loss = [trace1, trace2]

# Define the layout for the loss plot
layout_loss = go.Layout(title='Training Loss over Epochs', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss', type='log'))

# Define the data traces for the accuracy plot
trace1 = go.Scatter(x=np.arange(window_size//2, len(lr_acc_smooth)+window_size//2), y=lr_acc_smooth, name='Logistic Regression', mode='lines+markers')
trace2 = go.Scatter(x=np.arange(window_size//2, len(nn_acc_smooth)+window_size//2), y=nn_acc_smooth, name='Neural Network', mode='lines+markers')
data_accuracy = [trace1, trace2]

# Define the layout for the accuracy plot
layout_accuracy = go.Layout(title='Training Accuracy over Epochs', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy', tickformat=".2%"))

# Create the loss and accuracy plots
fig_loss = go.Figure(data=data_loss, layout=layout_loss)
fig_accuracy = go.Figure(data=data_accuracy, layout=layout_accuracy)

# Add a legend to the plots
fig_loss.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig_accuracy.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Show the plots
fig_loss.show()
fig_accuracy.show()



# Graphing with Plotly

I am using the loss and accuracy functions as a metric in plotly. I am doing some extra things, using a window size to control how many samples on either side of the current sample are used to calculate the weighted average. Some extra smoothing the loss afte that. Finally standard trace and plot functions.

Final Thoughts

The neural net performs better than the logistic regression model, but not by much. This was expected, by looking at the dataset it is easy to see patterns in the data that we can use to tell the flowers apart. The logistic regression model works just as good for this task even outperforming at some points along the way.